## Real-time ticker for Coffee commodity prices

*********************************************************************************
#Created at: 26/07/2017
#Author: Joshua Smith
#Purpose: Using a Url to generate a live ticker stream of coffee commodity prices
**********************************************************************************

In [1]:
#importing libraries to be used
import bs4 as bs 
from urllib.request import Request, urlopen
import pyodbc
from datetime import datetime
import time

In [74]:
#Connect to the SQL server through datasourcename path
conn = pyodbc.connect(r'DSN=starbucks')
curr= conn.cursor()

#Define a function
def priceinformation():
    req=Request(r'https://uk.investing.com/commodities/us-coffee-c', headers={'User-Agent': 'Mozilla/5.0'})
    
#Read through the URL which contains coffee commodity ticker      
    webpage = urlopen(req).read()
    s = bs.BeautifulSoup(webpage,'html.parser')
    
#Pass through the html and search for 'class' and the string of words 
    Priceline = s.find_all('span', {'class':'arial_26 inlineblock pid-8832-last'})
    Dateline = s.find_all('span', {'class': 'bold pid-8832-time'})

#for loop: iterate through the Dateline, if ':' doesn't exist a -1 will be produced.
#Therefore only continue the loop if this value is greater than zero

    for e in Dateline:
        codeline = str(e)
        if codeline.find(':')>0:
            position= codeline.find(':')
            Time =codeline[position -2: position +6]
            FinalTime =datetime.strptime(str(datetime.now()),'%Y-%m-%d %H:%M:%S.%f') #transform the string into datetime format
            daytime = 1 #daytime=1: daytime 
            
            
        else: 
            daytime=0  #daytime=0: nighttime 
       
        
    for e in Priceline:  #for loop: iterate through the Priceline 
        if daytime == 1:
            codeline =str(e)
            position = codeline.find('.')    #look for '.', is the dp in the price e.g 300.04 
            Price = codeline[position -3:position +3]
                                    
          
    if daytime == 1:   #Will only insert into sql table during daytime hours and when the market is open     
        curr.execute('insert into coffee (DateTim, [Price]) values(\'{0}\', \'{1}\')'.format(FinalTime, Price))
        conn.commit()
              

In [75]:
#while loop: loop through the function priceinformation, every 60 seconds
while True:
    priceinformation()
    time.sleep(60)

KeyboardInterrupt: 